In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from init_objects import *
from hft_model import *

# Baseline simulation

Simulate the baseline model dynamics to produce data on:

1. Price
2. Returns
3. Volume
4. Orderbook depth
5. Percentage of trades by HFT vs LFT
6. HFT profitability 

In [2]:
NRUNS = 10

In [5]:
parameters = {
    # global parameters
    "n_lft": 50,
    "n_hft": 5,
    "ticks": 1000,
    "fundamental_value": 10,
    "lft_sample_size": 0.2,
    "hft_sample_size": 0.6,
    "std_noise": 0.01,
    "std_LFT_price": 0.01,
    "std_LFT_vol": 4,
    "std_HFT_vol": 4,
    "return_on_investment": 0.5,
    "investment_frequency": 100,
    # lft parameters
    "w_fundamentalists": 1,
    "w_chartists": 10,
    "w_random": 1,
    # hft parameters
    "inventory_target": 20,
    "minimum_price_increment":0.1,
    "investment_fraction":0.4,
    # initial values
    "wealth": 300,
    "horizon_min": 1,
    "horizon_max": 4,
    "av_return_interval_max": 4,
    "init_price": 1,
    "agent_order_price_variability": (1,1),
    "total_hft_money": 200,
    "hft_speed": 1,
    "hft_init_investment": 0
}

## Run model

In [6]:
# 1 initalise model objects
high_frequency_traders, low_frequency_traders, orderbook = init_objects(parameters)

In [7]:
hfms = []
lfts = []
orderbooks = []
for seed in range(NRUNS): # optional, remove tqdm(), it is used for the progress bar
    hft, lft, ob = hft_model(high_frequency_traders, low_frequency_traders, orderbook, parameters, seed)
    hfms.append(hft)
    lfts.append(lft)
    orderbooks.append(ob)

## Retrieve data

In [10]:
average_tick_price = []
total_tick_volume = []
returns = []
autocorr_returns = []
absolute_returns = []
autocorr_abs_returns = []
for idx, orderbook in enumerate(orderbooks):
    average_tick_price.append([np.mean(prices) for prices in orderbook.transaction_prices_history])
    total_tick_volume.append([sum(volumes) for volumes in orderbook.transaction_volumes_history])
    returns.append(pd.Series(np.array(average_tick_price[idx])).pct_change())
    autocorr_returns.append([pd.Series(returns[idx]).autocorr(lag=lag) for lag in range(25)])
    absolute_returns.append(pd.Series(returns[idx]).abs())
    autocorr_abs_returns.append([absolute_returns[idx].autocorr(lag=lag) for lag in range(25)])